In [27]:
import requests
from bs4 import BeautifulSoup as bs
import json

In [58]:
URL="http://www.104.com.tw/i/apis/jobsearch.cfm?fmt=8&cols=JOB,\
    JOB_ADDR_NO_DESCRIPT,NAME,ADDRESS,PRODUCT,PROFILE,JOBCAT_DESCRIPT,\
    DESCRIPTION,APPEAR_DATE,SAL_MONTH_LOW,SAL_MONTH_HIGH,WELFARE,LAT,\
    LON,JOBSKILL_ALL_DESC&pgsz=2000&order=5&cat=2006001000&asc=1"
res=requests.get(URL+"&page=1")
res.status_code

200

In [59]:
data=json.loads(res.text)

In [61]:
data

{'PAGE': '1',
 'PAGECOUNT': '200',
 'RECORDCOUNT': '30238',
 'TOTALPAGE': '152',
 'data': [{'APPEAR_DATE': '20171026',
   'JOB': '【接案】果汁飲料調配與口味配方開發',
   'NAME': '104外包網'},
  {'APPEAR_DATE': '20171002', 'JOB': '【接案】雪花冰磚製作技術傳授', 'NAME': '104外包網'},
  {'APPEAR_DATE': '20171016', 'JOB': '【接案】專案營養師顧問', 'NAME': '104外包網'},
  {'APPEAR_DATE': '20171013', 'JOB': '【接案】冷凍調理食品研發', 'NAME': '104外包網'},
  {'ADDRESS': '建一路176號7樓之1',
   'APPEAR_DATE': '20171120',
   'JOB': '儲備幹部【大台北區】',
   'JOBCAT_DESCRIPT': '儲備幹部@餐飲服務生@類目',
   'JOBSKILL_ALL_DESC': '',
   'LAT': '0.0',
   'NAME': '瓦城 /  非常泰  / 1010 湘  /  大心  /  十食湘_瓦城泰統股份有限公司',
   'PRODUCT': '',
   'PROFILE': '瓦城泰統集團秉持著傳承東方廚藝、發揚東方美味的使命感，與永續經營的理念，\n將食材管理複雜且烹調技藝困難的東方菜系，\n透過「美食品牌化、經營連鎖化、管理科學化」的創新思維與管理，\n成為目前業界唯一能將東方菜系成功連鎖化、直營店數近百店的餐飲集團！ \n\n目前旗下共有五大品牌，包含全台第一大單一連鎖餐飲品牌的【瓦城泰國料理】、\n【非常泰概念餐坊】、【大心 新泰式麵食】、【1010湘】以及【十食湘BISTRO】，\n市場版圖跨及兩岸，並成功上櫃，\n已發展為台灣最大東方美食連鎖餐飲集團，並持續往國際舞台拓展。\n\n在屢創營運佳績的步調中，瓦城泰統規劃朝多元化發展，將陸續推出新品牌、新風格、新菜系，\n藉由我們的品牌，讓東方美味能夠發揚到國際，和更多人分享東方美味的感動，\n穩健邁向世界第

In [49]:
allpage=int(data["TOTALPAGE"])
allpage

152

In [65]:
HRdata104=[]
for i in range(1,allpage):
    res=requests.get(URL+"&page=%s"%i)
    if res.status_code==200:
        alldata=json.loads(res.text)
        HRdata104+=[case for case in alldata['data'] if len(case)>3]
        print(i)

        
        



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151


In [66]:
len(HRdata104)

30194

In [67]:
HRdata104[20]

{'ADDRESS': '農安街32號9樓',
 'APPEAR_DATE': '20171127',
 'JOB': '【營運部】侍酒師',
 'JOBCAT_DESCRIPT': '餐飲服務生@調酒師／吧台人員@國內業務人員',
 'JOBSKILL_ALL_DESC': '',
 'LAT': '25.091075',
 'NAME': '三井日本料理餐廳有限公司_三井餐飲事業集團',
 'PRODUCT': '時令鮮度與氛圍映心的餐飲服務，堅持品味用心生活，三井執著，會心期待。多年累積新鮮食材進口優勢，三井選品體現嚴選商品，饒富時令美昧延伸餐廳到家的無界服務。',
 'PROFILE': '自1992年,三井競競業業耕耘至今.如一行出遊的旅人,持續朝向內心羅盤的曙光前進,總在工作中懷抱專注的研究熱情,由細微觀察用心嚴選食材以符合自然健康趨勢. 遠赴海鮮產地, 直接採購急鮮運送提供給顧客最優質的食材.誠信負責是三井寶貴的品牌資產,顧客導向是三井價值的最高指標.我們相信執行力可以凝聚願力,不斷改善全力以赴。\n\n          發展至今三井已有十家門市,約900多位工作夥伴. 一路走來深深瞭解人才是三井最重要的資產, 具備優秀的員工,才能將真誠的服務美意及品牌精神忠實的傳達予客人,而不斷的標新創意是三井向前邁進的重要因素,時時注入新血,才能讓三井團隊保持豐沛的創新動力,人才的培育是需要長遠務實的規劃, 三井不斷延攬國際餐飲專業人才, 積極協助各餐飲學校專技培育研修,提供學生進入正規的專業餐飲領域。\n\n         夢想熱情是起點 , 堅持努力無終點 . 三井拓展的腳步不是汲於店數的快速擴張, 而是戮力於基石穩固, 專注研究入裡的服務精神 , 以創新需求引領趨勢 , 穩健務實的思維共創ㄧ方屬於你我所共築之大極殿堂 . 這些過程中, 三井不斷尋找志同道合的夥伴, 經由企業文化的感染熏陶及紮根實務的在職訓練 , 將原本來自四面八方背景迴異的人 , 在銜接的雨露均霑過程中 , 蛻變為未來都能被付予分店經營重任的進階主管. 透過公平透明的成果分享平臺, 使更多人創業的美夢, 能在三井儘早實現 . 三井追求進步的心 , 不斷前進未曾退縮 . 願景繁華似錦 , 唯有當下努力是真。',
 'SAL_MONTH_HIGH': '000000',
 'SAL_MONT

In [68]:
with open("./data/HRdata104","w") as f:
    json.dump(HRdata104,f)

In [69]:
import pymongo
from pymongo import MongoClient
import datetime
import json
import time
from functools import reduce

#連Mongodb
client = MongoClient('172.20.26.39', 27017)
#連DB名
db=client['rawData']
#秀DB內的collection list
db.collection_names()
#選擇我們想要的collection
collection = db.HRdata104
with open("./data/HRdata104") as f:
    HRdata104=json.load(f)
    print("讀取完畢")
collection.drop()
collection.insert_many(HRdata104)

讀取完畢


In [71]:
HRdata104=list(collection.find({}))

In [72]:
HRdata104[0]

{'ADDRESS': '立言街36號',
 'APPEAR_DATE': '20171123',
 'JOB': '洗碗作業員【台北地區】',
 'JOBCAT_DESCRIPT': '清潔工／資源回收人員@生鮮人員@洗碗人員',
 'JOBSKILL_ALL_DESC': '',
 'LAT': '25.091075',
 'NAME': '鼎泰豐小吃店股份有限公司',
 'PRODUCT': '中華點心美食。',
 'PROFILE': '※鼎泰豐精緻美味傳奇\n\n創立於1958年的鼎泰豐，在1972年由原來的食用油行轉型經營小籠包與麵點生意，開始賣小籠包後，致力於食材、品質與服務的提升，來自四面八方的佳評如潮，並在各家美食報章雜誌的介紹下，漸入佳績，至今不僅是一般市民的最愛，更是許多政商名流、國際級影星讚不絕口的頂級美食。\n1993年鼎泰豐獲紐約時報評選為全球十大特色餐廳之一，也是唯一入選的華人餐廳。2010~2014年香港鼎泰豐五度獲得米其林一顆星的評鑑，這是台灣餐飲業品牌的殊榮，台灣本店與世界各地的分店更是與有榮焉。2013年更榮獲美國 The Daily Meal網站選為亞洲最佳餐廳第一名殊榮，及CNN評為全球最佳連鎖企業第2名。\n鼎泰豐海外分店遍及日本、美國、韓國、新加坡、中國、香港、印尼、馬來西亞、澳洲、泰國、澳門等國家。鼎泰豐的美味，不僅征服台灣人的味蕾，更要讓全世界都能一同享受到這份來自台灣的經典美味。\n\n※The Legend of Din Tai Fung\n\nDin Tai Fung was originally founded as a cooking oil retail shop in 1958. As the consumption patterns of cooking oil started to change in the early 1970\'s threatening the survival of the business, Din Tai Fung was transformed to a restaurant in 1972 serving XiaoLongBao.\nThe New York Times, 1993, rated Din Tai Fung as among the top ten r

# 試跑多緒

In [73]:
import requests
from bs4 import BeautifulSoup as bs
import json
import concurrent.futures
import time

URL="http://www.104.com.tw/i/apis/jobsearch.cfm?fmt=8&cols=JOB,\
    JOB_ADDR_NO_DESCRIPT,NAME,ADDRESS,PRODUCT,PROFILE,JOBCAT_DESCRIPT,\
    DESCRIPTION,APPEAR_DATE,SAL_MONTH_LOW,SAL_MONTH_HIGH,WELFARE,LAT,\
    LON,JOBSKILL_ALL_DESC&pgsz=2000&order=5&cat=2006001000&asc=1"
res=requests.get(URL+"&page=1")
data=json.loads(res.text)
allpage=int(data["TOTALPAGE"])



def getdatalist(url,page):
    res=requests.get(url+"&page=%s"%page)
    if res.status_code==200:
        data=json.loads(res.text)
        datalist=[case for case in data['data'] if len(case)>3]
        print(page)
    return datalist


HRdata104=[]
start = time.time()
i=0
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    jobInfo = [executor.submit(getdatalist, URL, page) for page in range(1,allpage+1)]
    for future in concurrent.futures.as_completed(jobInfo):
#         try:
        data = future.result()
        HRdata104+=data
#         except Exception as e:
#             print(e)
#             print("zzz")
#             pass
        
end = time.time()  
print("-----------------")
print(end-start)


4
15
6
18
17
7
19
13
11
16
20
8
3
9
1
22
27
21
5
23
34
33
2
30
31
32
10
24
14
28
29
41
12
45
47
36
38
44
43
26
46
25
42
35
49
37
51
50
54
48
52
40
39
55
56
58
59
63
61
65
62
68
71
76
75
77
92
85
99
101
110
108


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [81]:
url="http://www.104.com.tw/i/apis/jobsearch.cfm?fmt=8&cols=JOB,\
    JOB_ADDR_NO_DESCRIPT,NAME,ADDRESS,PRODUCT,PROFILE,JOBCAT_DESCRIPT,\
    DESCRIPTION,APPEAR_DATE,SAL_MONTH_LOW,SAL_MONTH_HIGH,WELFARE,LAT,\
    LON,JOBSKILL_ALL_DESC&pgsz=2000&order=5&cat=2006001000&asc=1"
res=requests.get(url+"&page=152")
if res.status_code==200:
    data=json.loads(res.text)
    datalist=[case for case in data['data'] if len(case)>3]
data

{'PAGE': '15',
 'PAGECOUNT': '200',
 'RECORDCOUNT': '30239',
 'TOTALPAGE': '152',
 'data': [{'ADDRESS': '中和路212號3樓',
   'APPEAR_DATE': '20171123',
   'JOB': '餐廳儲備幹部-玫瑰聚',
   'JOBCAT_DESCRIPT': '餐飲服務生@門市／店員／專櫃人員@櫃檯接待人員',
   'JOBSKILL_ALL_DESC': '',
   'LAT': '25.0657724',
   'NAME': '玫瑰精品旅館股份有限公司',
   'PRODUCT': '玫瑰國際餐旅集團主要以精緻商旅.餐飲的經營與開發',
   'PROFILE': '玫瑰精品旅館是玫瑰國際餐旅集團下的子公司,母公司主要以旅館開發,餐飲經營與建築建設開發為主,旗下擁有兩家建設公司及5間精品旅館,在玫瑰國際餐飲集團的經營團隊下,也將集團事業採向多元化的經營與開發,在母公司細心灌溉下,旗下的公司也慢 慢的開花結果,這就是公司一直秉持著 穩健 踏實與創新的理想\n\n旅館部\n玫瑰精品旅館  Rose Boutique Hotel\n永不凋零的玫瑰象徵著企業永續的精神\n沉靜的紅色代表熱忱的心\n在踏實中不斷堅持的創新\n希望帶給顧客們雋永卻清新的享受\n\n高貴為花中之后的玫瑰\n意外地平易近人\n雖是容易看見的花種卻每每有著不同的感受\n這正是我們系列旅館的理念\n\n在平凡中 看見極致\n\n餐飲部 \n玫瑰聚 Rose To Table\n『人與人之間需要情感的相聚』\n玫瑰從誕生那一刻起\n即賦予相聚有了新的的意義\n\n§玫瑰。聚§\n玫瑰讓我們相聚有了藉口⋯\n學會愛人與被愛  珍惜了情感相聚的時光 \n\n這朵玫瑰，代表我們不變的愛✨\n我們相聚從這裡開始，與你們一起開始。',
   'SAL_MONTH_HIGH': '032000',
   'SAL_MONTH_LOW': '028000',
   'WELFARE': '◆ 獎金/禮品類\n1.年終獎金 \n2.三節獎金/禮品\n3.績效獎金\n◆ 保險類\n1.勞保\n2.健保\n3.員工團保\n◆ 餐飲休閒類\n1.定期聚餐\n2.伙食費補助\n◆ 制度類\n1